<a href="https://colab.research.google.com/github/TrelisResearch/install-guides/blob/main/8_bit_quantization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sharding of LLMs

The free environment of colab without a GPU has a large amount of RAM (around 25GB), which means you can use it to load models with large weights that may not fit in an environment with a GPU.

Built by Trelis. Find us on [HuggingFace](https://huggingface.co/Trelis).

### Login to Huggingface Hub

You need to create token with write role [here](https://huggingface.co/settings/tokens) and input in cell below.

In [ ]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
cache_dir=''

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


In [ ]:
# import os
# cache_dir = "/content/drive/My Drive/huggingface_cache"
# os.makedirs(cache_dir, exist_ok=True) # Ensure the directory exists


### Install requirements

In [ ]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install transformers==4.31 #temporary fix required owing to breaking changes on Aug 9th 2023

# !pip install -q einops
# !pip install -q sentencepiece


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip show bitsandbytes


Name: bitsandbytes
Version: 0.41.1
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


In [ ]:
!pip install -q -U xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 9.6 MB/s eta 0:00:00


### Model loading

In [ ]:
## Full precision models
# model_id = 'meta-llama/Llama-2-7b-chat-hf'
# model_id = 'mosaicml/mpt-7b-instruct'
# model_id = 'Trelis/Llama-2-7b-chat-hf-sharded-bf16'
# model_id = 'Trelis/Llama-2-13b-chat-hf-function-calling'

## 8-bit models
model_id = 'Trelis/Llama-2-7b-chat-hf-hosted-inference-8bit'
# model_id = 'Trelis/mpt-7b-instruct-hosted-inference'

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TextStreamer

# # MPT Models
# config = transformers.AutoConfig.from_pretrained(model_id, trust_remote_code=True)
# config.init_device = 'cuda:0' # Unclear whether this really helps a lot or interacts with device_map.
# config.max_seq_len = 512

# model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=True, config=config)

# Llama 2 Models
model = AutoModelForCausalLM.from_pretrained(model_id, device_map='auto')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
# MPT Inference
def stream(user_instruction):
    INSTRUCTION_KEY = "### Instruction:"
    RESPONSE_KEY = "### Response:"
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    PROMPT_FOR_GENERATION_FORMAT = """{intro}
    {instruction_key}
    {instruction}
    {response_key}
    """.format(
        intro=INTRO_BLURB,
        instruction_key=INSTRUCTION_KEY,
        instruction="{instruction}",
        response_key=RESPONSE_KEY,
    )

    prompt = PROMPT_FOR_GENERATION_FORMAT.format(instruction=user_instruction)

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda:0")
    streamer = TextStreamer(tokenizer)

    # Despite returning the usual output, the streamer will also print the generated text to stdout.
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=500, eos_token_id=0, temperature=1)

In [ ]:
#Llama 2 Inference
def stream(user_prompt):
    system_prompt = 'You are a helpful assistant that provides accurate and concise responses'

    B_INST, E_INST = "[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

    prompt = f"{B_INST} {B_SYS}{system_prompt.strip()}{E_SYS}{user_prompt.strip()} {E_INST}\n\n"

    inputs = tokenizer([prompt], return_tensors="pt").to("cuda:0")

    streamer = TextStreamer(tokenizer)

    # Despite returning the usual output, the streamer will also print the generated text to stdout.
    _ = model.generate(**inputs, streamer=streamer, max_new_tokens=500)

In [ ]:
stream('Count to ten')

<s> [INST] <<SYS>>
You are a helpful assistant that provides accurate and concise responses
<</SYS>>

Count to ten [/INST]

Of course! Here are the numbers from 1 to 10:

1
2
3
4
5
6
7
8
9
10</s>


### Saving to the hub

In [ ]:
save_model = f"Trelis/{model_id.split('/')[1]}-hosted-inference"

model.push_to_hub(save_model)
tokenizer.push_to_hub(save_model)

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:1708: UserWarning: You are calling `save_pretrained` to a 8-bit converted model you may likely encounter unexepected behaviors. If you want to save 8-bit models, make sure to have `bitsandbytes>0.37.2` installed.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/6.86G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Trelis/mpt-7b-instruct-hosted-inference/commit/9e32c4ef87d676b5cf467fdc166d5a41dc0d9528', commit_message='Upload tokenizer', commit_description='', oid='9e32c4ef87d676b5cf467fdc166d5a41dc0d9528', pr_url=None, pr_revision=None, pr_num=None)